# Pandas concat() tricks you should know to speed up your data analysis

Pandas provides various built-in functions for easily combining DataFrames. Among them, the concat() function seems fairly straightforward to use, but there are still many tricks you should know to speed up your data analysis.

In this article, you’ll learn Pandas `concat()` tricks to deal with the following common problems:
* Dealing with index and axis
*  Avoiding duplicate indices
*  Adding a hierarchical index with keys and names options
*  Column matching and sorting
*  Loading and concatenating datasets from a bunch of CSV files

In [1]:
import pandas as pd
df1 = pd.DataFrame({
    'name': ['A', 'B', 'C', 'D'],
    'math': [60,89,82,70],
    'physics': [66,95,83,66],
    'chemistry': [61,91,77,70]
})
df1

,name,math,physics,chemistry
0,A,60,66,61
1,B,89,95,91
2,C,82,83,77
3,D,70,66,70


In [2]:
df2 = pd.DataFrame({
    'name': ['E', 'F', 'G', 'H'],
    'math': [66,95,83,66],
    'physics': [60,89,82,70],
    'chemistry': [90,81,78,90]
})
df2

,name,math,physics,chemistry
0,E,66,60,90
1,F,95,89,81
2,G,83,82,78
3,H,66,70,90


##  Dealing with index and axis
Suppose we have 2 datasets about exam grades.

The simplest concatenation with concat() is by passing a list of DataFrames, for example[df1, df2]. And by default, it is concatenating vertically along the `axis 0` and preserving all existing indices.

In [3]:
pd.concat([df1,df2])

,name,math,physics,chemistry
0,A,60,66,61
1,B,89,95,91
2,C,82,83,77
3,D,70,66,70
0,E,66,60,90
1,F,95,89,81
2,G,83,82,78
3,H,66,70,90


If you want the concatenation to ignore existing indices, you can set the argument ignore_index=True. Then, the resulting DataFrame index will be labeled with 0, …, n-1.


In [4]:
pd.concat([df1, df2],ignore_index=True)

,name,math,physics,chemistry
0,A,60,66,61
1,B,89,95,91
2,C,82,83,77
3,D,70,66,70
4,E,66,60,90
5,F,95,89,81
6,G,83,82,78
7,H,66,70,90


To concatenate DataFrames horizontally along the axis 1 , you can set the argument axis=1 .


In [5]:
pd.concat([df1, df2], axis=1)

,name,math,physics,chemistry,name,math,physics,chemistry
0,A,60,66,61,E,66,60,90
1,B,89,95,91,F,95,89,81
2,C,82,83,77,G,83,82,78
3,D,70,66,70,H,66,70,90


## Avoiding duplicate indices

Now, we know that the concat() function preserves indices. If you’d like to verify that the indices in the result of pd.concat() do not overlap, you can set the argument verify_integrity=True. With this set to True, it will raise an exception if there are duplicate indices.

In [6]:
try:
    pd.concat([df1,df2], verify_integrity=True)
except ValueError as e:
    print('ValueError:', e)

ValueError: Indexes have overlapping values: Int64Index([0, 1, 2, 3], dtype='int64')


### Adding a hierarchical index with keys and names options
It is quite useful to add a hierarchical index (Also known as multi-level index) for more sophisticated data analysis. In this case, let’s add index Year 1 and Year 2 for df1 and df2 respectively. To do that, we can simply specify the keys argument.

In [7]:
res = pd.concat([df1, df2], keys=['Year 1','Year 2'])
res

name  math  physics  chemistry
Year 1 0    A    60       66         61
       1    B    89       95         91
       2    C    82       83         77
       3    D    70       66         70
Year 2 0    E    66       60         90
       1    F    95       89         81
       2    G    83       82         78
       3    H    66       70         90

In [10]:
res.loc['Year 2']

,name,math,physics,chemistry
0,E,66,60,90
1,F,95,89,81
2,G,83,82,78
3,H,66,70,90


In addition, the argument names can be used to add names for the resulting hierarchical index. For example: add name Class to the outermost index we just created.


In [11]:
pd.concat(
    [df1, df2], 
    keys=['Year 1', 'Year 2'],
    names=['Class', None],
)

name  math  physics  chemistry
Class                                  
Year 1 0    A    60       66         61
       1    B    89       95         91
       2    C    82       83         77
       3    D    70       66         70
Year 2 0    E    66       60         90
       1    F    95       89         81
       2    G    83       82         78
       3    H    66       70         90

To reset an index and turn it into a data column, you can use reset_index()


In [14]:
pd.concat(
    [df1, df2], 
    keys=['Year 1', 'Year 2'],
    names=['Class', None],
).reset_index(level=0)   
# reset_index(level='Class')

,Class,name,math,physics,chemistry
0,Year 1,A,60,66,61
1,Year 1,B,89,95,91
2,Year 1,C,82,83,77
3,Year 1,D,70,66,70
0,Year 2,E,66,60,90
1,Year 2,F,95,89,81
2,Year 2,G,83,82,78
3,Year 2,H,66,70,90


In [16]:
# Pass a string to level
pd.concat(
    [df1, df2], 
    keys=['Year 1', 'Year 2'],
    names=['Class', None],
).reset_index(level='Class')

,Class,name,math,physics,chemistry
0,Year 1,A,60,66,61
1,Year 1,B,89,95,91
2,Year 1,C,82,83,77
3,Year 1,D,70,66,70
0,Year 2,E,66,60,90
1,Year 2,F,95,89,81
2,Year 2,G,83,82,78
3,Year 2,H,66,70,90


## Columns matching and sorting
The concat() function is able to concatenate DataFrames with the columns in a different order. By default, the resulting DataFrame would have the same sorting as the first DataFrame. For example, in the following example, it’s the same order as df1.

If you prefer the resulting DataFrame to be sorted alphabetically, you can set the argument sort=True.

In [17]:
pd.concat([df1, df2], sort=True)

,chemistry,math,name,physics
0,61,60,A,66
1,91,89,B,95
2,77,82,C,83
3,70,70,D,66
0,90,66,E,60
1,81,95,F,89
2,78,83,G,82
3,90,66,H,70


In [18]:
# custom sort
custom_sort = ['math', 'chemistry', 'physics', 'name']
res = pd.concat([df1, df2])
res[custom_sort]

,math,chemistry,physics,name
0,60,61,66,A
1,89,91,95,B
2,82,77,83,C
3,70,70,66,D
0,66,90,60,E
1,95,81,89,F
2,83,78,82,G
3,66,90,70,H


## Loading and concatenating datasets from a bunch of CSV files

In [26]:
# Bad
import pathlib2 as pl2
ps = pl2.Path('.\\data\\d\\')
res = None
for p in ps.glob('*.csv'):
    if res is None:
        res = pd.read_csv(p)
    else:
        res = pd.concat([res, pd.read_csv(p)])
        
        
res.head()

,Unnamed: 0,Date,AveragePrice,Total Volume,4046,4225,4770,Total Bags,Small Bags,Large Bags,XLarge Bags,type,year,region
0,0,2015-12-27,0.49,1137707.43,738314.80,286858.37,11642.46,100891.80,70749.02,30142.78,0.0,conventional,2015,PhoenixTucson
1,1,2015-12-20,0.53,1097224.25,785254.94,204147.30,10346.68,97475.33,72169.92,25305.41,0.0,conventional,2015,PhoenixTucson
2,2,2015-12-13,0.66,907470.09,546182.56,241774.69,9429.99,110082.85,92028.11,18054.74,0.0,conventional,2015,PhoenixTucson
3,3,2015-12-06,0.56,1105500.34,760680.02,271207.14,13354.80,60258.38,60255.64,2.74,0.0,conventional,2015,PhoenixTucson
4,4,2015-11-29,0.75,724915.60,449043.34,208439.29,9770.44,57662.53,57662.53,0.00,0.0,conventional,2015,PhoenixTucson


In [28]:
import pathlib2 as pl2
ps = pl2.Path('.\\data\\d\\')

dfs = (
    pd.read_csv(p, encoding='utf8') for p in ps.glob('*.csv')
)
res = pd.concat(dfs )
res.head()

,Unnamed: 0,Date,AveragePrice,Total Volume,4046,4225,4770,Total Bags,Small Bags,Large Bags,XLarge Bags,type,year,region
0,0,2015-12-27,0.49,1137707.43,738314.80,286858.37,11642.46,100891.80,70749.02,30142.78,0.0,conventional,2015,PhoenixTucson
1,1,2015-12-20,0.53,1097224.25,785254.94,204147.30,10346.68,97475.33,72169.92,25305.41,0.0,conventional,2015,PhoenixTucson
2,2,2015-12-13,0.66,907470.09,546182.56,241774.69,9429.99,110082.85,92028.11,18054.74,0.0,conventional,2015,PhoenixTucson
3,3,2015-12-06,0.56,1105500.34,760680.02,271207.14,13354.80,60258.38,60255.64,2.74,0.0,conventional,2015,PhoenixTucson
4,4,2015-11-29,0.75,724915.60,449043.34,208439.29,9770.44,57662.53,57662.53,0.00,0.0,conventional,2015,PhoenixTucson


# How to do a Custom Sort on Pandas DataFrame
Take a look at the problem
Suppose we have a dataset about a clothing store:

In [32]:
import numpy as np

from pandas.api.types import CategoricalDtype

df = pd.DataFrame({
    'cloth_id': [1001, 1002, 1003, 1004, 1005, 1006],
    'size': ['S', 'XL', 'M', 'XS', 'L', 'S'],})

df.sort_values('size')

,cloth_id,size
4,1005,L
2,1003,M
0,1001,S
5,1006,S
1,1002,XL
3,1004,XS


We can see that each cloth has a size value and the data should be sorted by the following order:
- XS for extra small
- S for small
- M for medium
- L for large
- XL for extra large

However, you will get the following output when calling sort_values('size') .

In [33]:
df.sort_values('size')

,cloth_id,size
4,1005,L
2,1003,M
0,1001,S
5,1006,S
1,1002,XL
3,1004,XS


The output is not we want, but it is technically correct. Under the hood, sort_values() is sorting values by numerical order for number data or character alphabetically for object data.

Here are two common solutions:

1. Create a new column for custom sorting

2. Cast data to category type with orderedness using CategoricalDtype

## Create a new column for custom sorting.
In this solution, a mapping DataFrame is needed to represent a custom sort, then a new column will be created according to the mapping, and finally we can sort the data by the new column. Let’s see how this works with the help of an example.
Firstly, let’s create a mapping DataFrame to represent a custom sort.

In [35]:
df_mapping = pd.DataFrame({
    'size': ['XS', 'S', 'M', 'L', 'XL'],
})

sort_mapping = df_mapping.reset_index().set_index('size')
sort_mapping

,index
size,
XS,0
S,1
M,2
L,3
XL,4


After that, create a new column size_num with mapped value from sort_mapping

In [36]:
df['size_num'] = df['size'].map(sort_mapping['index'])
df['size_num']

0    1
1    4
2    2
3    0
4    3
5    1
Name: size_num, dtype: int64

In [37]:
df.sort_values('size_num')

,cloth_id,size,size_num
3,1004,XS,0
0,1001,S,1
5,1006,S,1
2,1003,M,2
4,1005,L,3
1,1002,XL,4


This certainly does our work. But it has created a spare column and can be less efficient when dealing with a large dataset.
We can solve this more efficiently using `CategoricalDtype`.

## Cast data to category type with orderedness using CategoricalDtype
CategoricalDtype is a type for categorical data with the categories and orderedness [1]. It is very useful for creating a custom sort [2]. Let’s see how this works with the help of an example.
Firstly, let’s import CategoricalDtype.


In [39]:
from pandas.api.types import CategoricalDtype

Then, create a custom category type cat_size_order with

the 1st argument set to ['XS', 'S', 'M', 'L', 'XL'] for the unique value of cloth size.
and the 2nd argument ordered=True for this variable to be treated as a ordered categorical.

In [41]:
cat_size_order = CategoricalDtype(
    ['XS', 'S', 'M', 'L', 'XL'], 
    ordered=True
)

cat_size_order

CategoricalDtype(categories=['XS', 'S', 'M', 'L', 'XL'], ordered=True)

After that, call astype(cat_size_order) to cast the size data to the custom category type. By running df['size'], we can see that the size column has been casted to a category type with the order [XS < S < M < L < XL].

In [43]:
df['size'] = df['size'].astype(cat_size_order)
df['size']

0     S
1    XL
2     M
3    XS
4     L
5     S
Name: size, dtype: category
Categories (5, object): [XS < S < M < L < XL]

In [45]:
df.sort_values('size')

,cloth_id,size,size_num
3,1004,XS,0
0,1001,S,1
5,1006,S,1
2,1003,M,2
4,1005,L,3
1,1002,XL,4


### View category codes property with the Series.cat accessor
Now the size column has been casted to a category type, and we could use Series.cat accessor to view categorical properties. Under the hood, it is using the category codes to represent the position in an ordered categorical.
Let’s create a new column codes, so we could compare size and codes values side by side.

In [46]:
df['codes'] = df['size'].cat.codes
df

,cloth_id,size,size_num,codes
0,1001,S,1,1
1,1002,XL,4,4
2,1003,M,2,2
3,1004,XS,0,0
4,1005,L,3,3
5,1006,S,1,1


We can see that XS, S, M, L, and XL has got a code 0, 1, 2, 3, 4, and 5 respectively. Codes are the positions of the actual values in the category type. By running df.info() , we can see that codes are int8.

In [47]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   cloth_id  6 non-null      int64   
 1   size      6 non-null      category
 2   size_num  6 non-null      int64   
 3   codes     6 non-null      int8    
dtypes: category(1), int64(2), int8(1)
memory usage: 436.0 bytes


### Sort by multiple variables
Next, let’s make things a little more complicated. Here, we’re going to sort our DataFrame by multiple variables.

In [52]:
df = pd.DataFrame({
    'order_id': [1001, 1002, 1003, 1004, 1005, 1006, 1007],
    'customer_id': [10, 12, 12, 12, 10, 10, 10],
    'month': ['Feb', 'Jan', 'Jan', 'Feb', 'Feb', 'Jan', 'Feb'],
    'day_of_week': ['Mon', 'Wed', 'Sun', 'Tue', 'Sat', 'Mon', 'Thu'],
})

# Similarly, let’s create 2 custom category types cat_day_of_week and cat_month, and pass them to astype().
df

,order_id,customer_id,month,day_of_week
0,1001,10,Feb,Mon
1,1002,12,Jan,Wed
2,1003,12,Jan,Sun
3,1004,12,Feb,Tue
4,1005,10,Feb,Sat
5,1006,10,Jan,Mon
6,1007,10,Feb,Thu


In [56]:
cat_day_of_week = CategoricalDtype(
    ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun'], 
    ordered=True
)
cat_month = CategoricalDtype(
    ['Jan', 'Feb', 'Mar', 'Apr'], 
    ordered=True,
)
df['day_of_week'] = df['day_of_week'].astype(cat_day_of_week)
df['month'] = df['month'].astype(cat_month)

In [54]:
df.sort_values(['month', 'day_of_week'])

,order_id,customer_id,month,day_of_week
5,1006,10,Jan,Mon
1,1002,12,Jan,Wed
2,1003,12,Jan,Sun
0,1001,10,Feb,Mon
3,1004,12,Feb,Tue
6,1007,10,Feb,Thu
4,1005,10,Feb,Sat


![](./i/1_f_8_NcC-AOW3aok87_lpLA.png )

In [55]:
df.sort_values(['customer_id', 'month', 'day_of_week'])

,order_id,customer_id,month,day_of_week
5,1006,10,Jan,Mon
0,1001,10,Feb,Mon
6,1007,10,Feb,Thu
4,1005,10,Feb,Sat
1,1002,12,Jan,Wed
2,1003,12,Jan,Sun
3,1004,12,Feb,Tue


![](./i/1_FdE185YWgUTeV1sXQ_MntA.png)

# All the Pandas merge() you should know for combining datasets
Pandas provides various built-in functions for easily combining datasets. Among them, merge() is a high-performance in-memory operation very similar to relational databases like SQL. You can use merge() any time when you want to do database-like join operations.
In this article, we’ll be going through some examples of combining datasets using Pandas merge() function. We will cover the following common usages and should help you get started with data combinations.
- The simplest call without any key column
- Specifying key columns using on
- Merging using left_on and right_on
- Various forms of joins: inner, left, right and outer
- Using validate to avoid invalid records

## Without any key column
When you use merge(), the simplest call must have two arguments: the left DataFrame and the right DataFrame. For example, to combine df_customer and df_info:

In [58]:
df_customer = pd.DataFrame({
    'id': [1, 2, 3, 4],
    'name': ['Tom', 'Jenny', 'James', 'Dan'],
})
df_info = pd.DataFrame({
    'id': [2, 3, 4, 5],
    'age': [31, 20, 40, 70],
    'sex': ['F', 'M', 'M', 'F']
})
pd.merge(df_customer, df_info)

,id,name,age,sex
0,2,Jenny,31,F
1,3,James,20,M
2,4,Dan,40,M


![](./i/1_H60kVHfgvZVZODkFTZksvw.png)

By default, the function will combine data on common columns (It is the column id in our example) and produces only the result that matches in both left and right DataFrames.
The following is an equivalent statement if you prefer to call merge from the left DataFrame.


In [59]:
df_customer.merge(df_info) 

,id,name,age,sex
0,2,Jenny,31,F
1,3,James,20,M
2,4,Dan,40,M


## Specifying key columns using argument on
You can specify the common columns for merging. To do so, pass an additional argument on as the name of the common column, here 'id' in our example, to merge() function:

In [64]:
# with multiple key columns
df_order = pd.DataFrame({
    'id': [2,3,4,5],
    'name': ['Jenny', 'James', 'Dan', 'leo'],
    'quantity': [2,4,6,10]
})


pd.merge(df_customer, df_order, on=['id','name'])


,id,name,quantity
0,2,Jenny,2
1,3,James,4
2,4,Dan,6


In [65]:
pd.merge(df_customer, df_info, on='id')

,id,name,age,sex
0,2,Jenny,31,F
1,3,James,20,M
2,4,Dan,40,M


If you use on , you have to make sure the column you specify must be present in both left and right DataFrames.
To combine data on multiple common columns, you can pass a list toon:


## Merging using left_on and right_on
It might happen that the column on which you want to merge the DataFrames have different names. For such merges, you will have to specify the left_on as the left DataFrame name and right_on as the right DataFrame name, for example:

In [67]:
df_info_2 = pd.DataFrame({
    'customer_id': [2,3,4,5],
    'age': [31,20,40,70],
    'sex': ['F', 'M', 'M', 'F']
})
df_info_2

,customer_id,age,sex
0,2,31,F
1,3,20,M
2,4,40,M
3,5,70,F


In [68]:
pd.merge(
  df_customer, 
  df_info_2, 
  left_on='id', 
  right_on='customer_id'
)

,id,name,customer_id,age,sex
0,2,Jenny,2,31,F
1,3,James,3,20,M
2,4,Dan,4,40,M


arious type of joins: inner, left, right and outer
They are 4 types of joins available to Pandas merge() function. The logic behind these joins is very much the same that you have in SQL when you join tables. You can perform a type of join by specifying the how argument with the following values:
- inner: the default join type in Pandas merge() function and it produces records that have matching values in both DataFrames
- left: produces all records from the left DataFrame and the matched records from the right DataFrame
- right: produces all records from the right DataFrame and the matched records from the left DataFrame
- outer: produces all records when there is a match in either left or right DataFrame
![](./i/1_3bL8ihJmwShib8Xj90X4Pw.png)

 ###  inner join
By default, Pandas merge() is performing the inner join and it produces only the set of records that match in both DataFrame.


![](./i/1_xd2B575qzZWi8FlzgrESpw.png)

In [72]:
df_customer = pd.DataFrame({
    'id': [1,2,3,4],
    'name': ['Tom', 'Jenny', 'James', 'Dan'],
})
df_info = pd.DataFrame({
    'id': [2,3,4,5],
    'age': [31,20,40,70],
    'sex': ['F', 'M', 'M', 'F']
})
pd.merge(df_customer, df_info, how='inner', on='id')


,id,name,age,sex
0,2,Jenny,31,F
1,3,James,20,M
2,4,Dan,40,M


### left join
The left join produces all records from the left DataFrame, and the matched records from the right DataFrame. If there is no match, the left side will contain NaN. You can set the argument how='left' to do left join:


![](./i/1_8zT68RD5jO-SybZye6Y2Fw.png)

In [73]:
pd.merge(df_customer, df_info, how='left', on='id')

,id,name,age,sex
0,1,Tom,NaN,NaN
1,2,Jenny,31.0,F
2,3,James,20.0,M
3,4,Dan,40.0,M


### right join
The right join produces all records from the right DataFrame, and the matched records from the left DataFrame. If there is no match, the right side will contain NaN. You can set the argument how='right' to do right join:

![](./i/1_-kckqogCjVfYAvrawDSWFg.png)

In [74]:
pd.merge(df_customer, df_info, how='right', on='id')

,id,name,age,sex
0,2,Jenny,31,F
1,3,James,20,M
2,4,Dan,40,M
3,5,NaN,70,F


### outer join
The outer join produces all records when there is a match in either left or right DataFrame. NaN will be filled for no match on either sides. You can set the argument how='outer' to do outer join:

![](./i/1_HPLSsnORsg910cwhkiM_YQ.png)

In [75]:
pd.merge(df_customer, df_info, how='outer', on='id')

,id,name,age,sex
0,1,Tom,NaN,NaN
1,2,Jenny,31.0,F
2,3,James,20.0,M
3,4,Dan,40.0,M
4,5,NaN,70.0,F


## Using validate to avoid invalid records
The result of merge() might have an increased number of rows if the merge keys are not unique. 
The argument validate takes one of the following values, so you can use it to validate different merge outputs.
- one_to_one or 1:1 : check if merge keys are unique in both left and right datasets.
- one_to_many or 1:m: check if merge keys are unique in left dataset.
- many_to_one or m:1: check if merge keys are unique in right dataset.
- many_to_many or m:m: allowed, but does not result in checks.

In [77]:
df_customer = pd.DataFrame({
    'id': [1,2,3,4],
    'name': ['Tom', 'Jenny', 'James', 'Dan'],
})
df_order_2 = pd.DataFrame({
    'id': [2, 2, 4, 4],
    'product': ['A', 'B' ,'A', 'C'],
    'quantity': [31, 21, 20,40],
    'date': pd.date_range('2019-02-24', periods=4, freq='D')
})

pd.merge(df_customer, df_order_2, how='left', on='id')

,id,name,product,quantity,date
0,1,Tom,NaN,NaN,NaT
1,2,Jenny,A,31.0,2019-02-24
2,2,Jenny,B,21.0,2019-02-25
3,3,James,NaN,NaN,NaT
4,4,Dan,A,20.0,2019-02-26
5,4,Dan,C,40.0,2019-02-27


![](./i/1_XbneAYYAvXUJ5FtYhCaESg.png)

Here are the reasons:

- the how='left' will produce all records from df_customer, and the matched records from df_order_2.
- In addition, the id in df_order_2 is not unique and all the matching records will be combined and returned.

This is an example of one-to-many merge. It is a valid scenario in our example, in which a customer can have multiple orders. However, one-to-many might be invalid in some other cases, for example, there are two records with the id value 2 in df_info

In [81]:
df_customer = pd.DataFrame({
    'id': [1, 2, 3, 4],
    'name': ['Tom', 'Jenny', 'James', 'Dan'],
})
df_info = pd.DataFrame({
    'id': [2, 2, 3, 4, 5],
    'age': [31, 21, 20, 40, 70],
    'sex': ['F', 'F', 'M', 'M', 'F']
})

pd.merge(df_customer, df_info, how='left', on='id')


,id,name,age,sex
0,1,Tom,NaN,NaN
1,2,Jenny,31.0,F
2,2,Jenny,21.0,F
3,3,James,20.0,M
4,4,Dan,40.0,M


This is certainly wrong because the same customer cannot have different information. To avoid this problem, we can set the argument validate to '1:1' , so it checks if merges keys are unique in both left and right DataFrames. It will raise a MergeError if the validation fails, for example:


In [84]:
pd.merge(df_customer, df_info, how='left', on='id', validate='1:1')

MergeError: Merge keys are not unique in right dataset; not a one-to-one merge

## Conclusion
Pandas merge() function is a simple, powerful, and high-performance in-memory operation very similar to relational databases like SQL.

I hope this article will help you to save time in combining datasets. I recommend you to check out the documentation for the merge() API and to know about other things you can do.